In [109]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.tree import plot_tree
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

In [110]:
np.random.seed(5)

In [123]:
speed1 = np.random.randint(0, 25, size=20000)
tmp_speed1 = np.random.randint(3, 25, size=10000)
tmp_speed2 = np.random.randint(0, 3, size=10000)


tilt1 = np.random.randint(200, 300, size=10000)
tilt2 = np.random.randint(0, 200, size=10000)

speed2 = np.concatenate((tmp_speed1, tmp_speed2))
tilt = np.concatenate((tilt1, tilt2))

In [124]:
data = []
for i in zip(speed1, speed2, tilt):
    data.append(list(i))

data = np.array(data)

In [127]:
label = []
for i in data:
    if i[0] >= 14 and i[1] <= 3 and i[2] <=200:
        label.append(1)
    else:
        label.append(0)
        
label = np.array(label)

In [128]:
datas = pd.DataFrame({
    'speed1':speed1,
    'speed2':speed2,
    'tilt':tilt,
    'label':label
})
datas

,speed1,speed2,tilt,label
0,5,15,298,0
1,5,9,225,0
2,6,3,208,0
3,16,24,230,0
4,18,6,222,0
...,...,...,...,...
19995,5,0,118,0
19996,16,1,87,1
19997,5,1,49,0
19998,1,2,13,0


In [129]:
x_train, x_test, y_train, y_test = train_test_split(data, label, 
                                                      test_size=0.2,
                                                      random_state=83,
                                                      shuffle=True,
                                                      stratify=label)

In [130]:
ss = StandardScaler()
ss.fit(x_train)
x_train_scaled= ss.transform(x_train)
x_test_scaled= ss.transform(x_test)

In [131]:
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

In [132]:
lr = LogisticRegression()
lr.fit(x_train_scaled, y_train)
scores = cross_validate(lr, x_train_scaled, y_train)
print(scores)

lr_pred=lr.predict(x_test_scaled)
print(accuracy_score(y_test, lr_pred))

{'fit_time': array([0.0100019 , 0.00900173, 0.01000214, 0.01000214, 0.00900173]), 'score_time': array([0., 0., 0., 0., 0.]), 'test_score': array([0.9621875, 0.9609375, 0.96625  , 0.9690625, 0.969375 ])}
0.9685


In [133]:
gs_dt= GridSearchCV(DecisionTreeClassifier(random_state=221), params, n_jobs=-1)
gs_dt.fit(x_train_scaled, y_train)
dt_clf = gs_dt.best_estimator_

dt_clf_pred=dt_clf.predict(x_test_scaled)
print(accuracy_score(y_test, dt_clf_pred))

1.0


In [134]:
gs_rf= GridSearchCV(RandomForestClassifier(random_state=221), params, n_jobs=-1)
gs_rf.fit(x_train_scaled, y_train)
rf_clf = gs_rf.best_estimator_

rf_clf_pred=rf_clf.predict(x_test_scaled)
print(accuracy_score(y_test, rf_clf_pred))

1.0


In [145]:
import time

In [150]:
temp = np.array([[15, 8, 223]])
temp_scaled=ss.transform(temp)

In [157]:
start = time.time()
lr_pred=lr.predict(temp_scaled)
end = time.time()

print(lr_pred)
print(f"{end - start:.100f} sec")

[0]
0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000 sec


In [158]:
start = time.time()
dt_clf_pred=dt_clf.predict(temp_scaled)
end = time.time()

print(dt_clf_pred)
print(f"{end - start:.100f} sec")

[0]
0.0000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000 sec


In [159]:
start = time.time()
rf_clf_pred=rf_clf.predict(temp_scaled)
end = time.time()

print(rf_clf_pred)
print(f"{end - start:.100f} sec")

[0]
0.0054085254669189453125000000000000000000000000000000000000000000000000000000000000000000000000000000 sec
